#Setup our parameters & keys

In [ ]:
import pandas as pd
import requests
import json
import numpy as np


api_id='api_id'
api_secret_key='api_secret'
org_key='org_key'
base_url='https://defense-prod05.conferdeploy.net'

#Alternatively you can prompt each time for information:
#api_id = input('Enter the CBC API ID: ')
#api_secret_key = input('Enter the CBC API Key: ')
#org_key = input('Enter the CBC Org Key: ')
#base_url = input('Enter the URL of your Carbon Black Cloud instance: ')

api_token= '/'.join([api_secret_key, api_id])

print('Your Parameters')
print('API ID: ' + api_id)
print('Secret: ' + api_secret_key)
print('Org key: ' + org_key)
print('Base URL: ' + base_url)
print('API token: ' + api_token)

#Get list of endpoints:

In [ ]:
#Form our request URL:
req_url = base_url + '/appservices/v6/orgs/' + org_key + '/devices/_search'

#Set our headers
headers = {'X-Auth-Token': api_token,
           'Content-Type': 'application/json'
           }
#Set our data (body) for the request
data = {"criteria":{"deployment_type":["ENDPOINT","WORKLOAD","VDI"]},"sort":[{"field":"last_contact_time","order":"DESC"}],"start":1,"rows":10000,"":""}

#Double check they're ok
print('Request URL: ' + req_url)
print('Headers: ', end="") 
print(headers)
print('Data: ', end="")
print(data)

In [ ]:
#Make the request
req = requests.post(req_url, headers=headers, json=data)
print('Status code: ' + str(req.status_code))


In [ ]:
devices_dict = req.json()
devices_dict['results']
devices = pd.DataFrame.from_dict(devices_dict['results'])
devices.set_index('device_owner_id', drop=True, inplace=True)

print('Total devices found: ', end="")
print(devices_dict.get('num_found'))
devices.head()

In [ ]:
#Cool. Let's export to CSV now
devices.to_csv('devices.csv')

#Get Alerts


In [ ]:
#Form our request URL:
req_url = base_url + '/appservices/v6/orgs/' + org_key + '/alerts/_search'

#Set our headers
headers = {'X-Auth-Token': api_token,
           'Content-Type': 'application/json'
           }
#Set our data (body) for the request
data = {"criteria":{"group_results":"false","minimum_severity":"1","category":["THREAT"],"workflow":["DISMISSED"],"alert_type":["CB_ANALYTICS"],"create_time":{"range":"all"}},"query":"","sort":[{"field":"create_time","order":"DESC"}],"start":0,"rows":10000}

#Double check they're ok
print('Request URL: ' + req_url)
print('Headers: ', end="") 
print(headers)
print('Data: ', end="")
print(data)

In [ ]:
#Make the request
req = requests.post(req_url, headers=headers, json=data)
print('Status code: ' + str(req.status_code))


In [ ]:
#Let's see what we've got
alerts_dict = req.json()

print('Total alerts found: ', end="")
print(alerts_dict.get('num_found'))
print('Total alerts available: ', end="")
print(alerts_dict.get('num_available'))

alerts_dict['results']
alerts = pd.DataFrame.from_dict(alerts_dict['results'])
alerts.head()

In [ ]:
#Cool. Let's export to CSV now
alerts.to_csv('alerts.csv')

In [ ]:
sev_summary = pd.DataFrame(alerts.pivot_table(index='severity',aggfunc='count').id).rename(columns = {'id':'Total_Alerts'})
sev_summary

In [ ]:
sev_summary.plot(kind='bar')

#Get Users

In [ ]:
#Form our request URL:
req_url = base_url + '/appservices/v6/orgs/' + org_key + '/users'

#Set our headers
headers = {'X-Auth-Token': api_token,
           'Content-Type': 'application/json'
           }

#Double check they're ok
print('Request URL: ' + req_url)
print('Headers: ', end="") 
print(headers)

In [ ]:
#Make the request
req = requests.get(req_url, headers=headers)
print('Status code: ' + str(req.status_code))


In [ ]:
#Let's see what we've got
users_dict = req.json()
users_dict['users']
users = pd.DataFrame.from_dict(users_dict['users'])
users

In [ ]:
#Due to the "DEPRECATED" values above, we need to look up each user ("principal") against a different API endpoint and see their "grant" (role)

# See here for more info: https://developer.carbonblack.com/reference/carbon-black-cloud/platform/latest/access-profiles-and-grants/#get-grant-of-a-principal

#For each login_id in users, look up the principal's grant:
for i, j in users.iterrows():
  print('login_id ' + str(j.login_id) + ' ', end="")
  
  #Form our request URL:
  req_url = base_url + '/access/v2/orgs/' + org_key + '/grants/psc:user:' + org_key + ':' + str(j.login_id)

  #Set our headers
  headers = {'X-Auth-Token': api_token,
           'Content-Type': 'application/json'
           }

  #Make the request
  req = requests.get(req_url, headers=headers)
  print('Status code: ' + str(req.status_code) + ' ', end="")

  #Let's see what we've got
  principal_dict = req.json()
  role=np.array(principal_dict['roles'])

  #Clean up the results
  role=str(role).replace("BETA_","")
  role=role.replace("psc:role::","")
  role=role.replace("[","")
  role=role.replace("]","")
  role=role.replace("'","")
  role=role.replace("_", " ")

  print(role)
  
  #Set the new role value in the users dataframe
  users.at[i,'role']=role
  print()

  
  

login_id 6299 Status code: 200 SUPER ADMIN

login_id 8747 Status code: 200 SUPER ADMIN

login_id 8791 Status code: 200 SUPER ADMIN

login_id 8808 Status code: 200 SUPER ADMIN

login_id 9547 Status code: 200 VIEW ALL

login_id 9569 Status code: 200 SUPER ADMIN

login_id 9948 Status code: 200 None

login_id 10037 Status code: 200 SUPER ADMIN

login_id 10727 Status code: 200 SUPER ADMIN

login_id 10728 Status code: 200 SUPER ADMIN

login_id 10796 Status code: 200 SUPER ADMIN

login_id 10878 Status code: 200 None

login_id 10971 Status code: 200 None

login_id 11285 Status code: 200 SUPER ADMIN

login_id 11567 Status code: 200 None

login_id 11698 Status code: 200 None



In [ ]:
#Let's see the real user roles, now updated in the dataframe
users

,org_key,admin_login_version,auth_method,org_admin_version,org_id,login_id,login_name,email,first_name,last_name,phone,contact_id,contact_version,role
0,7PESY63N,511,PASSWORD,0,1035,6299,bengoff@gmail.com,bengoff@gmail.com,Ben,Goff,(816)289-3767,271176,0,SUPER ADMIN
1,7PESY63N,285,PASSWORD,0,1035,8747,nkeller+jbg@carbonblack.com,nkeller+JBG@carbonblack.com,Noelle,Keller,,443668,0,SUPER ADMIN
2,7PESY63N,14,PASSWORD,0,1035,8791,jgoff+jbg@carbonblack.com,jgoff+JBG@carbonblack.com,Admin-Ben,Goff,,444303,0,SUPER ADMIN
3,7PESY63N,159,PASSWORD,0,1035,8808,jmcreynolds+jbg@carbonblack.com,jmcreynolds+jbg@carbonblack.com,John,McReynolds,,444916,0,SUPER ADMIN
4,7PESY63N,2,PASSWORD,0,1035,9547,nkeller@vmware.com,nkeller@vmware.com,Noelle,Keller,3072674717,553175,0,VIEW ALL
5,7PESY63N,83,PASSWORD,0,1035,9569,maso@vmware.com,maso@vmware.com,Mercedes,Aso,,558568,0,SUPER ADMIN
6,7PESY63N,7,PASSWORD,0,1035,9948,sakulkarni@vmware.com,sakulkarni@vmware.com,Sameer,Kulkarni,,647967,0,None
7,7PESY63N,330,PASSWORD,0,1035,10037,jaredn@vmware.com,jaredn@vmware.com,Jared,Nichols,,669035,0,SUPER ADMIN
8,7PESY63N,42,PASSWORD,0,1035,10727,smcgibbon+jbg@vmware.com,smcgibbon+jbg@vmware.com,Scott,McGibbon,,752838,0,SUPER ADMIN
9,7PESY63N,9,PASSWORD,0,1035,10728,brettf+jbg@vmware.com,brettf+jbg@vmware.com,Brett,Ford,,752848,0,SUPER ADMIN


In [ ]:
#Cool. Let's export to CSV now
users.to_csv('users.csv')

#Get USB Devices


In [ ]:
#Looks like a 2-step process, similar to looking up users.
# POST {cbc-hostname}/device_control/v3/orgs/{org_key}/devices/_search

#Form our request URL:
req_url = base_url + '/device_control/v3/orgs/' + org_key + '/devices/_search'

#Set our headers
headers = {'X-Auth-Token': api_token,
           'Content-Type': 'application/json'
           }
#Set our data (body) for the request
data = {"query":"","criteria":{},"sort":[{"field":"last_seen","order":"DESC"}],"start":0,"rows":10000}

#Double check they're ok
print('Request URL: ' + req_url)
print('Headers: ', end="") 
print(headers)
print('Data: ', end="")
print(data)

In [ ]:
#Make the request
req = requests.post(req_url, headers=headers, json=data)
print('Status code: ' + str(req.status_code))

In [ ]:
usb_devices_dict = req.json()
usb_devices_dict['results']
usb_devices = pd.DataFrame.from_dict(usb_devices_dict['results'])
usb_devices.set_index('id', drop=True, inplace=True)
usb_devices

In [ ]:
#Cool. Let's export to CSV now
usb_devices.to_csv('usb_devices.csv')